## Robinhood Orders with Current Profit/Loss Calculation
Parse robinhood orders API response, flatten to get list of all orders and enrich by adding stock symbol and name
Calculate profit/loss by grouping by stock symbol and removing fully sold stocks

In [1]:
import pandas as pd
import urllib.request, json

In [2]:
orders_raw = pd.read_json('orders_sample.json', orient='columns')
orders_raw = orders_raw['results']

In [3]:
orders_raw

0    {'updated_at': '2018-12-21T19:31:18.339622Z', ...
1    {'updated_at': '2018-12-21T19:30:23.297759Z', ...
2    {'updated_at': '2018-12-21T19:29:06.588381Z', ...
3    {'updated_at': '2017-08-10T14:15:46.907458Z', ...
4    {'updated_at': '2017-10-19T14:12:55.200328Z', ...
5    {'updated_at': '2017-08-10T14:19:23.318570Z', ...
Name: results, dtype: object

In [4]:
orders_normalize = pd.io.json.json_normalize(orders_raw, record_path=['executions'], 
                                        meta=['instrument', 'side'], 
                                        sep='_')
orders_normalize

,id,price,quantity,settlement_date,timestamp,instrument,side
0,12d7sfc5-4f40-4c02-be91-fc5260765af7,149.19990000,10.00000,2018-12-26,2018-12-21T19:31:18.099000Z,https://api.robinhood.com/instruments/1790dd4f...,buy
1,6d3f9f83-c62d-4da1-8321-40804bc4cc3a,75.46000000,15.00000,2018-12-26,2018-12-21T19:30:23.072000Z,https://api.robinhood.com/instruments/e6a6e495...,buy
2,87426fee-234b-4760-b50c-8539a8b58464,106.77720000,10.00000,2018-12-26,2018-12-21T19:29:06.327000Z,https://api.robinhood.com/instruments/7fcf5e7b...,buy
3,e39b50f4-3aea-4235-8459-ff082a5af70d,142.42000000,15.00000,2017-08-15,2017-08-10T14:15:40.820000Z,https://api.robinhood.com/instruments/1790dd4f...,buy
4,65695025-251b-43d9-8aaf-bbd899a4623e,26.31100000,75.00000,2017-10-23,2017-10-19T14:12:55.165000Z,https://api.robinhood.com/instruments/0dd811b3...,sell
5,3f5d00e3-3295-43f9-bbc6-2f053164901f,24.39900000,75.00000,2017-08-15,2017-08-10T14:19:23.285000Z,https://api.robinhood.com/instruments/0dd811b3...,buy


### Get instrument name and symbol by querying instrument API & Current quote

In [5]:
unique_instruments = pd.DataFrame(orders_normalize['instrument'].unique(), columns=['instrument'])
unique_instruments

,instrument
0,https://api.robinhood.com/instruments/1790dd4f...
1,https://api.robinhood.com/instruments/e6a6e495...
2,https://api.robinhood.com/instruments/7fcf5e7b...
3,https://api.robinhood.com/instruments/0dd811b3...


In [6]:
def get_instrument_name(instrument_url):
    with urllib.request.urlopen(instrument_url) as url:
        data = json.loads(url.read().decode())
        return data['simple_name'], data['symbol']

In [7]:
def get_current_quote(symbol):
    with urllib.request.urlopen('https://api.robinhood.com/quotes/'+symbol+'/') as url:
        data = json.loads(url.read().decode())
        return data['last_trade_price']

In [8]:
unique_instruments[['name', 'symbol']] = unique_instruments.apply(lambda x:get_instrument_name(x['instrument']), axis=1, result_type="expand")
unique_instruments

,instrument,name,symbol
0,https://api.robinhood.com/instruments/1790dd4f...,PowerShares QQQ,QQQ
1,https://api.robinhood.com/instruments/e6a6e495...,Vanguard US REIT fund,VNQ
2,https://api.robinhood.com/instruments/7fcf5e7b...,Digital Realty Trust,DLR
3,https://api.robinhood.com/instruments/0dd811b3...,Bank of America,BAC


In [9]:
unique_instruments['current_price'] = unique_instruments.apply(lambda x:get_current_quote(x['symbol']), axis=1, result_type="expand")
unique_instruments

,instrument,name,symbol,current_price
0,https://api.robinhood.com/instruments/1790dd4f...,PowerShares QQQ,QQQ,143.500000
1,https://api.robinhood.com/instruments/e6a6e495...,Vanguard US REIT fund,VNQ,71.740000
2,https://api.robinhood.com/instruments/7fcf5e7b...,Digital Realty Trust,DLR,102.410000
3,https://api.robinhood.com/instruments/0dd811b3...,Bank of America,BAC,22.730000


### All orders along with name & symbol

In [10]:
orders = pd.merge(orders_normalize, unique_instruments, how='inner', on='instrument')
orders = orders[['symbol','name','price','quantity','side','timestamp','settlement_date','current_price']]
orders['timestamp'] = pd.to_datetime(orders['timestamp']).dt.strftime('%Y-%m-%d %H:%M')
orders.rename(columns={'timestamp':'purchase_date'}, inplace=True)
orders

,symbol,name,price,quantity,side,purchase_date,settlement_date,current_price
0,QQQ,PowerShares QQQ,149.19990000,10.00000,buy,2018-12-21 19:31,2018-12-26,143.500000
1,QQQ,PowerShares QQQ,142.42000000,15.00000,buy,2017-08-10 14:15,2017-08-15,143.500000
2,VNQ,Vanguard US REIT fund,75.46000000,15.00000,buy,2018-12-21 19:30,2018-12-26,71.740000
3,DLR,Digital Realty Trust,106.77720000,10.00000,buy,2018-12-21 19:29,2018-12-26,102.410000
4,BAC,Bank of America,26.31100000,75.00000,sell,2017-10-19 14:12,2017-10-23,22.730000
5,BAC,Bank of America,24.39900000,75.00000,buy,2017-08-10 14:19,2017-08-15,22.730000


### Calculate Profit/Loss

In [11]:
# Update quanity to negative, if side is sell
orders['quantity'] = orders.apply(lambda x: -float(x['quantity']) if x['side']=='sell' else float(x['quantity']), axis=1)
# Cost = Purchase cost if buy in +ve or Sell amount if sell in -ve
orders['cost'] = orders.apply(lambda x: (float(x['quantity'])*float(x['price'])), axis=1, result_type="expand")

In [12]:
# Calculate profit from original acquisition cost and current price
order_profit = orders.groupby('symbol').agg({'quantity':sum, 'cost':sum, 'purchase_date':'min', 'current_price':max})
order_profit.rename(columns={'purchase_date':'early_purchase_date'}, inplace=True)
order_profit = order_profit[order_profit.quantity >0]
order_profit['current_equity'] = order_profit.apply(lambda x: (float(x['quantity'])*float(x['current_price'])), axis=1, result_type="expand")
order_profit['profit'] = order_profit.apply(lambda x: (float(x['current_equity'])-float(x['cost'])), axis=1, result_type="expand")

In [13]:
# Calculate Equity Ratio
total_equity = order_profit['current_equity'].sum()
order_profit['equity_ratio'] = order_profit.apply(lambda x: (float(x['current_equity'])/float(total_equity))*100, axis=1, result_type="expand")

In [14]:
print('Total Equity:'+str(total_equity))
order_profit.sort_values(by=['equity_ratio'], ascending=False)

Total Equity:5687.700000000001


,quantity,cost,early_purchase_date,current_price,current_equity,profit,equity_ratio
symbol,,,,,,,
QQQ,25.0,3628.299,2017-08-10 14:15,143.500000,3587.5,-40.799,63.074705
VNQ,15.0,1131.900,2018-12-21 19:30,71.740000,1076.1,-55.800,18.919774
DLR,10.0,1067.772,2018-12-21 19:29,102.410000,1024.1,-43.672,18.005521


### Export data as CSV

In [15]:
order_profit.to_csv('order_profit.csv', encoding='utf-8', index=True)